In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)


def score_dataset(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score

# Prepare data
df = pd.read_csv("ames.csv")

/var/folders/yk/j514dw3d6hqdb24n2tpbqyx00000gn/T/ipykernel_15502/1326098923.py:10: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


In [3]:
df

MSSubClass                  MSZoning  \
0     One_Story_1946_and_Newer_All_Styles   Residential_Low_Density   
1     One_Story_1946_and_Newer_All_Styles  Residential_High_Density   
2     One_Story_1946_and_Newer_All_Styles   Residential_Low_Density   
3     One_Story_1946_and_Newer_All_Styles   Residential_Low_Density   
4                Two_Story_1946_and_Newer   Residential_Low_Density   
...                                   ...                       ...   
2925                  Split_or_Multilevel   Residential_Low_Density   
2926  One_Story_1946_and_Newer_All_Styles   Residential_Low_Density   
2927                          Split_Foyer   Residential_Low_Density   
2928  One_Story_1946_and_Newer_All_Styles   Residential_Low_Density   
2929             Two_Story_1946_and_Newer   Residential_Low_Density   

      LotFrontage  LotArea Street            Alley            LotShape  \
0           141.0  31770.0   Pave  No_Alley_Access  Slightly_Irregular   
1            80.0  11622.0   Pave  No_Alley_Access             Regular   
2            81.0  14267.0   Pave  No_Alley_Access  Slightly_Irregular   
3            93.0  11160.0   Pave  No_Alley_Access             Regular   
4            74.0  13830.0   Pave  No_Alley_Access  Slightly_Irregular   
...           ...      ...    ...              ...                 ...   
2925         37.0   7937.0   Pave  No_Alley_Access  Slightly_Irregular   
2926          0.0   8885.0   Pave  No_Alley_Access  Slightly_Irregular   
2927         62.0  10441.0   Pave  No_Alley_Access             Regular   
2928         77.0  10010.0   Pave  No_Alley_Access             Regular   
2929         74.0   9627.0   Pave  No_Alley_Access             Regular   

     LandContour Utilities LotConfig  ... PoolArea   PoolQC            Fence  \
0            Lvl    AllPub    Corner  ...      0.0  No_Pool         No_Fence   
1            Lvl    AllPub    Inside  ...      0.0  No_Pool  Minimum_Privacy   
2            Lvl    AllPub    Corner  ...      0.0  No_Pool         No_Fence   
3            Lvl    AllPub    Corner  ...      0.0  No_Pool         No_Fence   
4            Lvl    AllPub    Inside  ...      0.0  No_Pool  Minimum_Privacy   
...          ...       ...       ...  ...      ...      ...              ...   
2925         Lvl    AllPub   CulDSac  ...      0.0  No_Pool     Good_Privacy   
2926         Low    AllPub    Inside  ...      0.0  No_Pool  Minimum_Privacy   
2927         Lvl    AllPub    Inside  ...      0.0  No_Pool  Minimum_Privacy   
2928         Lvl    AllPub    Inside  ...      0.0  No_Pool         No_Fence   
2929         Lvl    AllPub    Inside  ...      0.0  No_Pool         No_Fence   

     MiscFeature  MiscVal MoSold YearSold SaleType  SaleCondition  SalePrice  
0            NaN      0.0      5     2010      WD          Normal     215000  
1            NaN      0.0      6     2010      WD          Normal     105000  
2           Gar2  12500.0      6     2010      WD          Normal     172000  
3            NaN      0.0      4     2010      WD          Normal     244000  
4            NaN      0.0      3     2010      WD          Normal     189900  
...          ...      ...    ...      ...      ...            ...        ...  
2925         NaN      0.0      3     2006      WD          Normal     142500  
2926         NaN      0.0      6     2006      WD          Normal     131000  
2927        Shed    700.0      7     2006      WD          Normal     132000  
2928         NaN      0.0      4     2006      WD          Normal     170000  
2929         NaN      0.0     11     2006      WD          Normal     188000  

[2930 rows x 79 columns]

In [10]:
df.columns 

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', 'FirstFlrSF', 'SecondFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF',

The k-means algorithm is sensitive to scale. This means we need to be thoughtful about how and whether we rescale our features since we might get very different results depending on our choices. As a rule of thumb, if the features are already directly comparable (like a test result at different times), then you would *not* want to rescale. On the other hand, features that aren't on comparable scales (like height and weight) will usually benefit from rescaling. Sometimes, the choice won't be clear though. In that case, you should try to use common sense, remembering that features with larger values will be weighted more heavily.

# 1) Scaling Features

Consider the following sets of features. For each, decide whether:
- they definitely should be rescaled,
- they definitely should *not* be rescaled, or
- either might be reasonable

Features:
1. `Latitude` and `Longitude` of cities in California
2. `Lot Area` and `Living Area` of houses in Ames, Iowa
3. `Number of Doors` and `Horsepower` of a 1989 model car

Once you've thought about your answers, write it below and proceed running the other cell below.

#Write your answer here: 
1.) Either might be reasonable
2.) Definitely should be rescaled
3.) Definetely should be rescaled

-------------------------------------------------------------------------------

# 2) Create a Feature of Cluster Labels

Creating a k-means clustering with the following parameters:
- features: `LotArea`, `TotalBsmtSF`, `FirstFlrSF`, `SecondFlrSF`,`GrLivArea`
- number of clusters: 10
- iterations: 10

(This may take a moment to complete.)

In [4]:
X = df.copy()
y = X.pop("SalePrice")


# YOUR CODE HERE: Define a list of the features to be used for the clustering
features = ["LotArea", "TotalBsmtSF", "FirstFlrSF", "SecondFlrSF", "GrLivArea"]


# Standardize
X_scaled = X.loc[:, features]
X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)


# YOUR CODE HERE: Fit the KMeans model to X_scaled and create the cluster labels
kmeans = KMeans(n_clusters=10, max_iter=10, random_state=0)
X["Cluster"] = kmeans.fit_predict(X_scaled)


/Users/apple/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


You can run the below cell to see the result of the clustering, if you like.

In [6]:
Xy = X.copy()
Xy["Cluster"] = Xy.Cluster.astype("category")
Xy["SalePrice"] = y
sns.relplot(
    x="value", y="SalePrice", hue="Cluster", col="variable",
    height=4, aspect=1, facet_kws={'sharex': False}, col_wrap=3,
    data=Xy.melt(
        value_vars=features, id_vars=["SalePrice", "Cluster"],
    ),
);

/Users/apple/anaconda3/lib/python3.11/site-packages/seaborn/axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)


And as before, `score_dataset` will score your XGBoost model with this new feature added to training data.

In [7]:
score_dataset(X, y)

0.13946376259429677

-------------------------------------------------------------------------------

The k-means algorithm offers an alternative way of creating features. Instead of labelling each feature with the nearest cluster centroid, it can measure the distance from a point to all the centroids and return those distances as features.

# 3) Cluster-Distance Features

Now add the cluster-distance features to your dataset. You can get these distance features by using the `fit_transform` method of `kmeans` instead of `fit_predict`.

In [8]:
kmeans = KMeans(n_clusters=10, n_init=10, random_state=0)


# YOUR CODE HERE: Create the cluster-distance features using `fit_transform`
X_cd = kmeans.fit_transform(X_scaled)


# Label features and join to dataset
X_cd = pd.DataFrame(X_cd, columns=[f"Centroid_{i}" for i in range(X_cd.shape[1])])
X = X.join(X_cd)

Run this cell to score these new features, if you like.


In [9]:
score_dataset(X, y)

0.1361233246230215